# Abschlussprojekt: Kiva funding platform

## CRISP DM: Business Understanding

Unser Geschäftsmodell ist das Betreiben einer Plattform (crowd-investing) bei der sich Personen die eine Geschäftsidee haben, aber nicht das benötigte Geld, anmelden und für ihr Projekt innerhalb einer vorgegebenen Zeit Geld sammeln können. 

Auf der anderen Seite haben wir Geldgeber, die gern ihr Geld in Projekte anlegen möchten und nach Investitionen suchen. 

Als Vermittler bringt unsere Plattform also Geldnehmer und Geldgeber zusammen. 

Unsere Datenbasis ist die Historie eurer Plattform. 

**Getroffene Annahmen zu unserem Geschäftsmodell**

Alle Projekte sind abgeschlossene Projekte, d.h. die Zeit, um für sein Projekt Geld zu sammeln ist abgelaufen. 
Unser Geschäftsmodell sieht es vor, dass die gesammelten Gelder ausgezahlt werden, auch wenn der Zielbetrag nicht erreicht wurde.

Wir verdienen unser Geld mit einer Provision für jedes Projekt was auf unserer Plattform landet. 
Wir nehmen an, dass wir in der Regel einen prozentualen Anteil bekommen und dass wir mit mehr Volumen aus den Projekten auch mehr Provision erhalten. D.h. das ein Mehr an Projekten oder teurere Projekte für uns von Vorteil sind.

Ziel ist daher die Geschäftserweiterung.

## CRISP DM: Data Understanding


    - funded_amount ... mit Ablauf der "Crowding"zeit erhaltener Betrag/ ausgezahlter Betrag in USD
    - loan_amount ... Zielbetrag (Betrag dem man für das Projekt erreichen wollte) in USD
    - activity ... Unterkategory zu dem das Ziel des Crowdprojektes thematisch gehört
    - sector ... Oberkategory in den das Crowdprojektes Thema fällt
    - use ... Kurzbeschreibung wofür das Geld verwendet werden soll
    - country_code ... Ländercode nach ISO Norm
    - country ... Ländername nach ISO Norm
    - region ... Region
    - currency ... Währung in den der funded_amount dann ausgezahlt wurde
    - term in months ... Dauer über die der Kredit ausgezahlt werden soll
    - lender_count ...Darlehensgeber (also wieviele Personen Geld für das Projekt gegeben haben)
    - borrower_genders ... Geschlecht und Anzahl der Darlehensnehmer, also diejenigen die das Crowdprojekt initiiert haben       
    - repayment interval ... vertraglich vereinbarte Rückzahlungsmodalitäten/-rhythmus

## Datenaufbereitung

### Bibs importieren

In [ ]:
import numpy as np
import pandas as pd
import plotly_express as px

# für Dashboard
from dash import Dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from dash import no_update

# misc
import re

# Wichtig, wenn das Notebook exportiert wird in HTML, dann werden die Grafiken auch im HTML angezeigt 
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

#recommened behaviour for pandas. Avoids warning.
pd.options.mode.copy_on_write = True

### Daten einlesen

In [ ]:
df = pd.read_csv("data_abschlussprojekt.csv", engine='python', nrows=2)
df

In [ ]:
df.columns

In [ ]:
df = pd.read_csv("data_abschlussprojekt.csv", 
                 sep='#',
                 engine="python", 
                 skipinitialspace=True, 
                 index_col=0)
df

### Objekte kennenlernen
                                #### Beobachtung: Problem: "[True, u'to start a turducken farm.']" 

In [ ]:
df.columns # Spalten

In [ ]:
print(df.info()) # DatenTypen 

### DatenTypen Konvertieren

In [ ]:
# wieviel unique values haben die object spalten
cols = df.columns
for col in cols:
    print(f"{col}: ", df[col].nunique())

In [ ]:
# potentielle categories
# gibt es verdächtige strings?
cols = ["sector", "country_code", "currency", "repayment_interval"]
# lists zu lang: "activity", "country"

for col in cols:
    strings = []
    for element in df[col].unique():
        strings.append(element)

    # Ausgabe der wirklichen Strings
    print(f"{col}: {strings}")

In [ ]:
df['activity'] = df['activity'].astype("category")
df['sector'] = df['sector'].astype("category")
df['use'] = df['use'].astype("string")
df['country_code'] = df['country_code'].astype("category")
df['country'] = df['country'].astype("category")
df['currency'] = df['currency'].astype("category")
df['region'] = df['region'].astype("string")
df['borrower_genders'] = df['borrower_genders'].astype("string")
df['repayment_interval'] = df['repayment_interval'].astype("category")

df.dtypes

### Duplicate

In [ ]:
df_duplicated = df[df.duplicated(keep=False)]
df_duplicated

- 34930 rows duplicated
--> diese **nicht Löschen** weil sie **nicht eindeutig** zu zuordnen sind. Es fehlen beispielsweise eine Identifikation und ein Antragsdatum.

### systemseitig Fehlende Werte - NaN

**1) fehlende Werte in 'use' und 'borrower_genders' verwefen,**

**2) fehlende Werte (8) im countrycode ersetzen unter Zuhilfenahme der Information durch die Spalte 'region'**

**3) ersetze 'region' NaNs mit einem Synonym z.B. "not specified"**

In [ ]:
df.isna().sum() # NaNs overview

#### fehlende Werte in 'use' und 'borrower_genders' verwerfen,
**Begründung:**
- da gleiche rows betroffen sind. Ebenfalls sind diese rows auch in 'region' NaNs
- kein weiteres Muster erkennbar, kein eindeutiges skewing feststellbar und sehr geringer Anteil am Datensatz (<1%)

In [ ]:
df.loc[df["use"].isna(), :  ] # NaNs in 'use' auflisten

In [ ]:
(df['use'].isna() & df['borrower_genders'].isna()).value_counts() # Anzahl der NaNs in 'borrower_gender' & 'use' sharing the same row indices? 3888

In [ ]:
# Anzahl der NaNs in 'borrower_gender' & 'use' & 'region' sharing the same row indices? Skewing? spezielles land betroffen? 3888
dfisna = df.loc[(df['use'].isna() & df['borrower_genders'].isna()
                 & df['region'].isna()), :]
dfisna

In [ ]:
dfisna['repayment_interval'].unique()

In [ ]:
dfisna['region'].unique()

In [ ]:
df = df.dropna(subset=['use']) # Löschen von Zeilen, in denen "use" NaN ist
df = df.dropna(subset=['borrower_genders']) # Löschen von Zeilen, in denen "borrower_genders" NaN ist

#### Nan in country Code für land Nambiba mit 'Na' befüllen
**Begründung:** 
- wir kennen den country code von Namibia

In [ ]:
dfisna2 = df[df['country_code'].isna()] # alle NaNs in 'countrycode' sind in 'country' Namibia (countrycode=Na)
dfisna2

In [ ]:
df['country_code'] = df['country_code'].cat.add_categories("Na") # neue Kategorie "Na" zu Category 'Country_code'

df.loc[:, 'country_code'] = df['country_code'].fillna("Na") # "Na" eintragen
na_country = df[df['country_code'] == "Na"]

na_country



#### Umgang mit Fehlende Werten in der Spalte 'region', ersetzen mit 'Not specified'

**Begründung:**
- region hat für die weitere Betrachtung der Oberkategorien (Sektoren) keinen großen Einfluss -> jedoch das Löschen der gesamten Zeilen könnte eine Verfälschung der Analyse zur Folge haben (56800 Datensätze (~9 %) betroffen).


In [ ]:
df.loc[:, 'region'] = df['region'].fillna("Not specified") # Auffuellen mit "Not specified"
df[df['region'] == "Not specified"]  # check

### Extremwerte

In [ ]:
df.describe()

In [ ]:
df[df.funded_amount == 100000]

In [ ]:
# Commented as it slows down notebook
# fig12 = px.box(df.funded_amount, title='Extremwerte in funded_amount?')
# fig12.update_layout(yaxis_type="log")
# fig12.show()

In [ ]:
df[df.loan_amount == 100000]

In [ ]:
fig13 = px.box(df.loan_amount, title='Extremwerte in loan_amount?')
fig13.update_layout(yaxis_type="log")
fig13.show()

In [ ]:
df[df.lender_count == 2986]

In [ ]:
df.sort_values(by='lender_count', ascending=False)

In [ ]:
# Commented as it slows down notebook
# fig14 = px.box(df.lender_count, 
#                title='Extremwerte in lender_count?'
#               )
# fig14.update_layout(yaxis_type="log")
# fig14.show()

#### Ergebnis Ausreißer/Extremwerte
Aus der Beschreibung des df (.describe()) ergeben sich potentielle Extremwerte/Ausreißer in der jeweils gleichen Zeile fuer funded_amount, loan_amount und lender_count (70499). Es handelt sich um einen Extremwert fuer ein Projekt mit 2986 beteiligten (aus lender_count)

### Zusätzliche KPIs
#### Funding Rate
Als zusätzliche KPI führen wir die Spalte funding_rate ein. Diese gibt an, wie hoch der Grad der Finanzierung in prozent ist, indem der funded_amount mit dem loan amount ins Verhaeltnis gesetzt wird.

In [ ]:
df['funding_rate'] = round((df['funded_amount'] / df['loan_amount']) * 100, 2)
df.head(10)

In [ ]:
sortedbyfr_df = df.sort_values(by='funding_rate', ascending=True)
sortedbyfr_df

In [ ]:
print(df.info())

In [ ]:
# number of uniques and unique items in column sector
print(df.loc[:,'sector'].nunique(), df.loc[:,'sector'].unique())

In [ ]:
# number of uniques and unique items in column activity
print(df.loc[:,'activity'].nunique(), df.loc[:,'activity'].unique())

#### Anzahl der am Projekt beteiligten Personen
Als zusätzliche KPI führen wir die Spalte borrower_count ein. Diese gibt an, wie viele Beteiligte (Darlehensnehmer) es am Projekt gab. Dazu zählen wir die Häufigkeit von "male" und "female" in der Spalte borrowers_gender

In [ ]:
df.loc[:,'borrower_genders'].unique()

In [ ]:
# Method to count persons (male, females) in list

def person_count(text):
    male_pattern = re.compile(r'\bmale\b', flags=re.IGNORECASE)
    male_count = len(male_pattern.findall(text))
    female_pattern = re.compile(r'\bfemale\b', flags=re.IGNORECASE)
    female_count = len(female_pattern.findall(text))
    return male_count + female_count


# Insert new row "borrower_count"
df['borrower_count'] = df['borrower_genders'].map(person_count)

## EDA

### Wer sind wir?
- wir sind Kiva, wir machen X funding moeglich in total per year / overall

In [ ]:
sumFundsTotal = df.loc[:,'funded_amount'].sum() # Summe der Fundings overall
sumFundsTotal

In [ ]:
sumLoansTotal = df.loc[:,'loan_amount'].sum() # Summe der Loans overall
sumLoansTotal

In [ ]:
fundingCapacityNom = sumLoansTotal-sumFundsTotal
fundingCapacityNom

In [ ]:
fundingCapacityRatio = sumFundsTotal/sumLoansTotal
fundingCapacityRatio

In [ ]:
#Anzahl der Projekte (donut fuer die 15 categories)
df_sector = df.groupby(by='sector', as_index=False, observed=False).agg(amount_of_projects=("sector","count")).sort_values(by="amount_of_projects", ascending=False)
df_sector = df_sector.reset_index(drop=True)
df_sector


In [ ]:
bottom_seven = df_sector.tail(7)
bottom_seven_sum = bottom_seven['amount_of_projects'].sum()
df_other = pd.DataFrame({'sector': ['Other'], 'amount_of_projects': [bottom_seven_sum]})
df_sector2 = pd.concat([df_sector, df_other], ignore_index=True).sort_values(by="amount_of_projects", ascending=False)
df_sector2 = df_sector2.drop(df_sector2.tail(7).index)
df_sector2 = df_sector2.reset_index(drop=True)
df_sector2

### In welchen Oberkategorien sind die meisten Projekte?

In [ ]:
#color map for our sectors
sectors = df["sector"].unique()
colorMap = {val: 'lightgrey' for val in sectors}

ourSectors = ["Entertainment", "Wholesale", "Health", "Construction", "Manufacturing"]
for val in ourSectors:
    colorMap[val]="yellow"
# zusätzlich noch Other Kategorie
colorMap["Other"] = "yellow"

In [1]:
df_sector = df.groupby(by='sector', as_index=False, observed=False).agg(amount_of_projects=(
    "sector", "count")).sort_values(by="amount_of_projects", ascending=False)
df_sector = df_sector.reset_index(drop=True)
df_sector

bottom_five = df_sector.tail(5)
bottom_five_sum = bottom_five['amount_of_projects'].sum()
df_other = pd.DataFrame(
    {'sector': ['Other'], 'amount_of_projects': [bottom_five_sum]})
df_sector2 = pd.concat([df_sector, df_other], ignore_index=True).sort_values(
    by="amount_of_projects", ascending=False)
df_sector2 = df_sector2.drop(df_sector2.tail(5).index)
df_sector2 = df_sector2.reset_index(drop=True)
df_sector2


df_sector4 = df_sector2.sort_values(
    by="amount_of_projects", ascending=True).reset_index(drop=True)
df_sector4

gewuenschte_reihenfolge = [
    "Other",
    "Transportation",
    "Arts",
    "Personal Use",
    "Education",
    "Housing",
    "Clothing",
    "Services",
    "Retail",
    "Food",
    "Agriculture",
]


fig1 = px.pie(df_sector4,
              values='amount_of_projects',
              names='sector',
              title='Großteil der Projekte beschäftigt sich mit der Befriedigung existentieller Beduerfnisse',
              hover_name='sector',
              color="sector",
              color_discrete_map=colorMap,
              category_orders={"sector": gewuenschte_reihenfolge},
              # sort=True  # Deaktivieren Sie die automatische Sortierung
              )
fig1.update_traces(textposition='inside',
                   textinfo='label',
                   hole=0.4,
                   direction='counterclockwise',
                   rotation=65,
                   selector=dict(type='pie')
                   )
annotation_text = "Health            1.42%<br>Construction   0.96%<br>Manufacturing 0.96%<br>Entertainment 0.13%<br>Wholesale       0.10%"

fig1.add_annotation(
    text=annotation_text,
    x=0.88,
    y=0.55,
    ax=0.51,
    ay=0.51,
    showarrow=True,
    arrowhead=2,
    font=dict(size=14),
    align="left"
)


fig1.add_annotation(text=' ', x=0.725, y=0.65, ax=38.0, ay=0,
                    showarrow=True, arrowhead=2, arrowwidth=2.0, font={"size": 14})
fig1.show()

NameError: name 'df' is not defined

##### Analyse
Fig.1: Anteilige Verteilung der Gesamtzahl der beantragten Projekte anhand ihrer Oberkategorien (Sektoren)

###### deskriptiv
Das Donut Diagramm zeigt einen Kreis, der in Bereiche unterteilt ist, wobei jeder Bereich eine bestimmte Oberkategorie repräsentiert, wie z.B. Agriculture, Food, Retail usw. Jeder Sektor wird proportional zu seinem Anteil an der Gesamtzahl der beantragten Projekte dargestellt.
Je groesser der Wert, desto mehr Antraege wurden in dem jeweiligen Sektor gestellt. Sektoren die weniger als 2% der Projekte ausmachen (Wholesale, Entertainment, Manufacturing, Construction) wurden zu Other zusammengefasst, gelb hervorgehoben und in Textform beschrieben.

###### analytisch

**Ermittlung des Sollzustandes** 

Alle Sektoren sollte gleichmaessig verteilt sein, weil wir ueberall ueber den Umsatz der Projekte und somit unsere Provison Geld verdienen wollen.

**Vergleich Ist - Sollzustand** 

Ein Großteil der Projekte beschäftigt sich mit der Befriedigung existentieller Beduerfnisse, Agriculture, Food und Retail sind die groessten Sektoren und machen zusammen ca. 60% aus. In den Bereichen die unter "Other" zusammengefasst sind besteht Wachstumspotential. 

**Limitation**

Die Verteilung der Zielbeträge in den jeweiligen Oberkategorien, sowie die tatsaechliche Funding rate gehen aus der Darstellung nicht hervor.

### Welche Oberkategorien haben höchste Zielbetragsgröße

In [ ]:
df_grouped3 = df.groupby(by=["sector"], as_index=False, observed=False).agg(mean_loan_amount=(
    "loan_amount", "mean")).sort_values(by="mean_loan_amount", ascending=False)
df_grouped3.reset_index()

fig2 = px.bar(df_grouped3,
             x="sector",
             y="mean_loan_amount",
             color='sector',
             color_discrete_map=colorMap,
             labels={"mean_loan_amount": "Median der Zielbeträge in USD",
                     "sector": "Oberkategorie"},
             title='Vier unserer fokussierten Bereiche haben größere mittlere Zielbeträge als der Median über alle Projekte',
             template="plotly_white",
             )

median = df_grouped3["mean_loan_amount"].median()
fig2.add_hline(y=median, line_width=2, line_dash="dash")

fig2.add_annotation(text='Mittlerer Zielbetrag über Median über alle Projekte',
                   x=1.3, y=1500, ax=10, ay=-40, showarrow=True, arrowhead=2, arrowwidth=2.0)

fig2.show()

#### Analyse
##### deskriptiv
- Achsen:
    - Die x-Achse zeigt die Oberkategorien der Projekte als Balken. Die Kategorien sind nach absteigenden Median der Zielbeträge innerhalb der Oberkategorien sortiert.
    - Die y-Achse zeigt den Median der Zielbeträge in USD.
- Die Balkenhöhe zeigt den Median der Zielbeträge innerhalb der jeweiligen Oberkategorie.
- Die gestrichelte Linie zeigt den Median der Zielbeträge über alle Projekte.


##### analytisch
**Sollzustand** 

Für uns als Unternehmen ist es wünschenswert, wenn auch die häufigsten Oberkategorien (siehen oben: "Agriculture", "Food", "Retail"), die höchsten Projekt-Zielwerte haben, damit wir eine hohe Anzahl an Projekten mit hohen Zielwerten und damit Provisionen haben.

Die Trennlinie oben ist der Median der Zielwerte aller Projekte. Alle Oberkategorien, die oberhalb liegen haben einen höheren Zielbetrag.

**Istzustand**

Die häufigsten Oberkategorien  ("Agriculture", "Food", "Retail") haben im Gegensatz zu unserem Sollzustand einen geringen mittleren Zielbetrag (Median) als der Median über alle Projekte.
Vier von fünf unserer fokussierten Oberkategorien (Health, Construction, Manufacturing, Entertainment, Wholesale) haben einen höheren mittleren Zielbetrag als der Median über alle Projekte.
Ein Wachstum in diesen fokussierten Oberkategorien hat damit auch aus Sicht der höheren Zielwerte der Projekte Vorteile, da wir als Unternehmen durch höhere Provisionen stärker von diesen Oberkategorien profitieren. 

**Limitationen**

Der Plot zeigt nur die Mediane innerhalb der Oberkategorien, wir sehen aber nicht die Streuung der Zielbeträge. Wir sehen auch nicht die Summe der Zielbeträge innerhalb der Oberkategorien, um anhand dieses Wertes zu sehen wie ausbaufähig die Oberkategorie ist.

### Wie hoch sind die Zielbetragsgrößen in den Kategorien nach Anzahl der am Projekt beteiligten Personen?

In [ ]:
# Only select Country Peru
df_selected = df.loc[df["country"] == "Peru", :]

# categorize this column into buckets 1-4, 5-9, ... Persons
bucketsize = 5
mapRangeToString = {0: "1-4", 5: "5-9", 10: "10-14",
                    15: "15-19", 20: "20-24", 25: "25-30"}
df_selected.loc[:, 'borrower_countCategory'] = (
    df_selected.loc[:, 'borrower_count'].copy() // bucketsize)*bucketsize
df_selected.loc[:, 'borrower_countCategory'] = df_selected.loc[:,
                                                               'borrower_countCategory'].copy().map(mapRangeToString)

df_grouped = df_selected.groupby(by=["sector", "borrower_countCategory"], observed=False).agg(
    meadian_loan=("loan_amount", "median"))

categoriesSortedByTotalSum = ["Agriculture", "Food", "Arts", "Retail", "Education", "Construction", "Construction",
                              "Manufacturing", "Clothing", "Entertainment", "Transportation", "Personal Use", "Services", "Health", "Housing", "Wholesale"]

df_matrix = df_grouped.pivot_table(
    index='borrower_countCategory', columns='sector', values='meadian_loan', aggfunc='median')
df_matrix
# Sort columns by given oder of categoriesSortedByTotalSum list
df_matrix_sorted = df_matrix.loc[:, categoriesSortedByTotalSum]

# Sort rows by given order of mapRangeToString to have categories 1-4, 5-9, ... in matrix sorted descending
borrowerOder = list(mapRangeToString.values())
borrowerOder.reverse()
df_matrix_sorted = df_matrix_sorted.loc[borrowerOder, :]

fig3 = px.imshow(df_matrix_sorted,
                 color_continuous_scale="YlOrRd",
                 labels={"y": "Anzahl der am Projekt Beteiligten",
                         "x": "Oberkategorie der Projekte",
                         "color": "Median der Zielbeträge in USD"},
                 title="Die möglichen Wachstums-Bereiche haben keine Projekte mit mehr als 20 Personen",
                 text_auto=True,
                 template="simple_white",
                 height=800)

fig3.add_hline(y=1.49, line_width=2, line_dash="dash",
               line_color="black", opacity=1.0)
fig3.add_vline(x=3.5, line_width=2, line_dash="dash",
               line_color="black", opacity=1.0)

# highlight insight that focused sections have less then 20  project participants
fig3.add_annotation(text='Unsere fokussierten Bereiche haben alle weniger als 20 Beteiligte',
                    x=8.0, y=1.0, showarrow=False, font={"size": 20})
for x in [5, 6, 8, 12, 14]:
    fig3.add_annotation(text='', x=x, y=1.45, showarrow=True,
                        arrowhead=2, arrowcolor="sandybrown", arrowwidth=2.0)

# highlight insight that Arts and Retail have lower median in project loans in category 20-25 people than in category 15-19 people
fig3.add_annotation(text='Zielbetrag ist weniger als bei 15-19 Beteiligten',
                    x=2.0, y=0.45, ax=0.0, showarrow=True, arrowhead=2, arrowwidth=2.0)
fig3.add_annotation(text=' ', x=3.0, y=0.45, ax=0.0, showarrow=True,
                    arrowhead=2, arrowwidth=2.0, font={"size": 15})

fig3.show()

##### Analyse

###### Deskriptiv
- Achsen:
    - Die x-Achse zeigt die Oberkategorien der Projekte (sectors). Die Oberkategorien sind nach der Summe der Zielbetragswerte in den Feldern der Heatmap der jeweiligen Oberkategorie absteigend sortiert. 
    - Die y-Achse zeigt die Anzahl der am Projekt Beteiligten in fünfer Schritten (borrower_count). Je höher der Wert, desto höher ist die Anzahl der Beteiligten in den Projekten.

- Farbskala: Die Farben repräsentieren die Höhe der Zielbeträge (Median der Zielbeträge in der Oberkategorie in USD) in verschiedenen Farbtönen von Gelb nach Rot. Dabei steht Rot für höhere Beträge und Gelb für niedrigere Beträge. Ein dunkelrotes Feld in einer bestimmten Zelle bedeutet, dass Projekte dieser Kategorie mit dieser Anzahl von Beteiligten tendenziell höhere Zielbeträge haben.


###### Analytisch
**Sollzustand**

Mit zunehmender Personenanzahl sollte sich auch die Projektgröße (in Form des Zielbetrags) erhöhen. Dies ist vorteilhaft für unser Unternehmen, da damit größere Projekte entstehen, die mehr Provision erbringen und weniger Verwaltungsaufwand bedeuten. Gleichzeitig ist es für Geldgeber einfacher, wenn sie die Auswahl aus wenigen großen Projekte haben, als vielen kleinen. Eine hohe Anzahl an Beteiligten kann den Geldgebern auch mehr Verlässlichkeit und Sicherheit vermitteln.

Die Aufteilung des Diagramms in 4 Quadranten erfolgt für Oberkategorien mit mehr/weniger als 20 beteiligten Personen pro Projekt. (horizontal: mehr/weniger als 20 Beteiligte, vertikal: Rechts stehen Oberkategorien mit weniger als 20 Beteiligten, Links davon die mit mehr als 20 Beteiligten)

**unterer Bereich (weniger als 20 Beteiligte, unterer linker und rechter Quadrant)**

Dieser Bereich entspricht unserem Sollzustand: nit steigender Personenanzahl nimmt auch der Median des Zielbetrags der Projekte zu.

**Oberer linker Quadrant**

In diesem Quadranten sind wieder die bereits als häufig identifizieren Oberkategorien Agriculture, Food, Retail vertreten. Es ist positiv, dass diese häufigen Oberkategorien auch eine hohe Spanne bei der Anzahl der am Projekt beteiligten Personen haben. Auffällig ist, dass Arts und Retail bei der Personenanzahl von 20-24 Personen geringere Zielwerte (4937 bzw. 4775 USB) haben, als bei Projekten mit 15-19 Beteiligten (5625 bzw. 5912 USD). In diesen Oberkategorien steckt noch Potenzial, große Projekte mit mehr Kreditvolumen zu initiieren.

**Oberer rechter Quadrant**

In diesem Quadranten sind alle Oberkategorien, die keine Projekte mit mehr als 20 Personen haben (es existieren keine eingefärbten Felder). In diesem Quadranten sind auch unsere fokussierten Oberkategorien (Health, Construction, Manufacturing, Entertainment, Wholesale). Dieser Quadrant bietet die Möglichkeit, die Menschen in diesem Land (Peru) zu größeren Projekten zu motivieren. Unser Unternehmen kann davon profitieren, indem sowohl die Anzahl der Projekte als auch die Zielbeträge erhöht werden, was zu höheren Provisionen führt. Gleichzeitig haben solche größeren Projekte den Vorteil, dass sie im Gegensatz zu vielen kleinen Projekten womöglich weniger Verwaltungsaufwand haben, als auch mehr Sicherheit suggerieren und auf mehr Investitionsbreitschaft bei den Geldgebern stoßen.

**Limitationen**

Der Graph gibt nicht die Häufigkeit oder Summe der Zielwerte in den Feldern wieder. Wir erkennen nur, dass es in dem oberen rechten Quadranten gar keine Projekte mit den entsprechenden Eigenschaften gibt. Wir erkennen aber nicht, ob es andere Felder gibt, die eine sehr geringe Häufigkeit bzw. Summe der Zielwerte haben und damit ebenfalls ausbaufähig sind.

## Dashboard

In [ ]:
# 1. App erstellen
my_app = Dash(__name__)

# 2. Layout erstellen
# Überschrift
my_app.layout = html.Div(children=[
    html.H1(children="Sektoren je Land", style={"textAlign": "center"}),
    # Daten auswählen
    dcc.Dropdown(
        id="graph_drop",
        options=[{'label': country, 'value': country}
                 for country in df['country'].unique()],
        value="Afghanistan",
        style={"width": "260px"}
    ),
    # Grafik
    dcc.Graph(id="theGraph", figure={}, style={
              'width': '800px', 'height': '600px'})
])

# 3. Callback erstellen


@my_app.callback(
    # 1. IMMER die Outputs
    Output(component_id="theGraph", component_property="figure"),

    # 2. IMMER die Inputs
    Input(component_id="graph_drop", component_property="value"),
)
def update_graph(selected_country):
    df_select = df[df["country"] == selected_country]
    gruppierte_daten = df_select.groupby(by="sector",  observed=False).size().reset_index(name="count")

    donut_chart = px.pie(gruppierte_daten, values="count", names="sector", title=f"Sektoren in {selected_country}",
                         labels={"sector": "Sektor", "count": "Anzahl"},
                         hole=0.4,
                         color="sector",
                         color_discrete_map={
                             "Agriculture": "#B4CF66",
                             "Food": "#44803F",
                             "Retail": "#AE65CF",
                             "Other": "#8A818F",
                             "Services": "#E0D424",
                             "Personal Use": "#F2C305",
                             "Housing": "#A67458",
                             "Clothing": "#88D6D6",
                             "Education": "#5C90B5",
                             "Entertainment": "#F24405",
                             "Wholesale": "#B87FE0",
                             "Arts": "#00B3AD",
                             "Transportation": "#575153",
                             "Health": "#3FA9F5",
                             "Construction": "#E06F24",
                             "Manufacturing": "#8856D8"
                         }
                         )

    return donut_chart


# 4. App starten
my_app.run_server(jupyter_mode="external", port=8106)

## Zusätzliche Analysen

In [ ]:
fig4 = px.bar(df_sector,
              y='amount_of_projects',
              x='sector',
              title='Distribution of projects',
              color='sector',
              color_discrete_sequence=px.colors.qualitative.Set2
              )
fig4.show()

**in bar chart weitere y achse einfuegen loan volume ?
**donut zu explosion mit weiterem ranking der laender? oder Verteilung in Antragstellerzahl, volumen etc.




'''
df_sectors_sorted = df.groupby(by="sector", as_index=False).agg( count = ("loan_amount", "mean") )
df_sectors_sorted = df_sectors_sorted.sort_values(by="count",ascending=False) 
sectors_sorted = df_sectors_sorted["sector"].to_list()

df_grouped2 = df.groupby(by=["sector"],as_index=False ).agg( count_loan_amount=("loan_amount","mean") )
fig = px.bar( df_grouped, x="sector", category_orders={"sector":sectors_sorted}, y="count_loan_amount")
fig.show()
df_sectors_sorted
'''

weiterer plot fuer die finanzielle hoehe der loans pro category

### KPI 

**funding rate

In [ ]:
fig1 = px.pie(df_sector, 
              values='amount_of_projects', 
              names='sector', 
              title='Distribution of projects',
              color_discrete_sequence=px.colors.qualitative.Set2
             )
fig1.update_traces(textposition='inside', textinfo='label+percent')
fig1.show()

In [ ]:
df.sort_values(by='funding_rate', ascending=False).head(10)

In [ ]:
# funding rate bins - welche sectors?

df['funding_rate_bin'] = pd.cut(df['funding_rate'], bins=3, labels=['Low', 'Medium', 'High'])
grouped = df.groupby(['funding_rate_bin', 'sector']).size().reset_index(name='count')
top_categories = grouped.groupby('funding_rate_bin').apply(lambda x: x.nlargest(5, 'count')).reset_index(drop=True)
print(top_categories)

In [ ]:
df_grouped2 = df.groupby(by=["sector"],as_index=False ).agg( sum_loan_amount=("loan_amount","sum") ).sort_values(by="sum_loan_amount", ascending=False)
fig3 = px.bar(df_grouped2, 
             x="sector",
             y="sum_loan_amount",
             color='sector',
             color_discrete_sequence=px.colors.qualitative.Set2
            )
fig3.show()
df_grouped2.reset_index()